In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
df = pd.read_csv("D:/deep learning/crime section recommandation system/ipc_sections.csv")

In [7]:
df.shape

(444, 4)

In [8]:
df.head()

,Description,Offense,Punishment,Section
0,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,IPC_140
1,Description of IPC Section 127\nAccording to s...,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property,IPC_127
2,Description of IPC Section 128\nAccording to s...,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine,IPC_128
3,Description of IPC Section 129\nAccording to s...,Public servant negligently suffering prisoner ...,Simple Imprisonment 3 Years + Fine,IPC_129
4,Description of IPC Section 130\nAccording to s...,"Aiding escape of, rescuing or harbouring, such...",Imprisonment for Life or 10 Years + Fine,IPC_130


In [27]:
df.isna().sum()

Description    0
Offense        0
Punishment     0
Section        0
dtype: int64

In [20]:
df_recovered = df.copy()

In [26]:
df['Offense'] = df['Offense'].fillna('')
df['Punishment'] = df['Punishment'].fillna('')

In [30]:
df['text'] = df['Offense'].fillna('') + ' ' + df['Description'].fillna('')

In [31]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['text'])


In [32]:
def recommend_ipc_sections(query, top_n=5):
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarity.argsort()[-top_n:][::-1]
    return df.iloc[top_indices][['Section', 'Offense', 'Punishment']]

In [33]:
query = "helped a prisoner escape from jail"
results = recommend_ipc_sections(query)
print(results)

     Section                                            Offense  \
3    IPC_129  Public servant negligently suffering prisoner ...   
2    IPC_128  Public servant voluntarily allowing prisoner o...   
4    IPC_130  Aiding escape of, rescuing or harbouring, such...   
124  IPC_223  Escape from confinement negligently suffered b...   
84   IPC_224  Resistance or obstruction by a person to his l...   

                                          Punishment  
3                 Simple Imprisonment 3 Years + Fine  
2           Imprisonment for Life or 10 Years + Fine  
4           Imprisonment for Life or 10 Years + Fine  
124  Simple Imprisonment for 2 Years or Fine or Both  
84                           2 Years or Fine or Both  


In [34]:
import tkinter as tk
from tkinter import ttk, scrolledtext

In [36]:
def get_recommendations():
    user_query = entry.get("1.0", tk.END).strip()
    if not user_query:
        result_text.delete("1.0", tk.END)
        result_text.insert(tk.END, "Please enter a description.")
        return
    results = recommend_ipc_sections(user_query)
    result_text.delete("1.0", tk.END)
    for i, row in results.iterrows():
        result_text.insert(tk.END, f"Section: {row['Section']}\n")
        result_text.insert(tk.END, f"Offense: {row['Offense']}\n")
        result_text.insert(tk.END, f"Punishment: {row['Punishment']}\n")
        result_text.insert(tk.END, "-"*50 + "\n")

In [37]:
root = tk.Tk()
root.title("Crime Section Recommender System")


''

In [38]:
tk.Label(root, text="Enter Crime Description:", font=('Arial', 12)).pack(pady=5)

In [39]:
entry = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=60, height=6)
entry.pack(padx=10, pady=5)

In [40]:
tk.Button(root, text="Recommend IPC Sections", command=get_recommendations, font=('Arial', 11), bg='lightblue').pack(pady=10)

In [41]:
result_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=70, height=15)
result_text.pack(padx=10, pady=5)


In [42]:
root.mainloop()